In [2]:
import argparse
import gzip, os, csv
import numpy as np
import random
import time

# word2vec learning

In [3]:
!pip install gensim --user

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import gensim
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            #print (fname)
            for line in open(os.path.join(self.dirname, fname)):
                yield line.lower().split()

In [5]:
sentences = MySentences('people/') # a memory-friendly iterator

In [7]:
model = gensim.models.Word2Vec(size=200, workers=8, window=5, sg=1, negative=15, iter=5)
model.build_vocab(sentences)


In [8]:
corpus_count = model.corpus_count
model.train(sentences, total_examples=corpus_count, epochs =5)

(253795, 445045)

In [10]:
model.wv.most_similar('hotel')

/opt/conda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('casino', 0.9981303215026855),
 ('number', 0.9958279132843018),
 ('estate', 0.9951328635215759),
 ('organization', 0.9949161410331726),
 ('compared', 0.9937310814857483),
 ('called', 0.9934476613998413),
 ('wearing', 0.9932417869567871),
 ('golf', 0.9931994676589966),
 ('told', 0.9927423596382141),
 ('pay', 0.9926958084106445)]

# RDF2Vec training

In [11]:
## create data structure for knowledge graph
def addTriple(net, source, target, edge):
    if source in net:
        if  target in net[source]:
            net[source][target].add(edge)
        else:
            net[source][target]= set([edge])
    else:
        net[source]={}
        net[source][target] =set([edge])
            
def getLinks(net, source):
    if source not in net:
        return {}
    return net[source]

# Generate paths (entity->relation->entity) by radom walks
def randomWalkUniform(triples, startNode, max_depth=5):
    next_node =startNode
    path = str(startNode)+'->'
    for i in range(max_depth):
        neighs = getLinks(triples,next_node)
        #print (neighs)
        if len(neighs) == 0: break
        weights = []
        queue = []
        for neigh in neighs:
            for edge in neighs[neigh]:
                queue.append((edge,neigh))
        edge, next_node = random.choice(queue)
        path = path +str(edge)+'->'
        path = path +str(next_node)+'->'
    path =path.split('->')
    return path

In [12]:
# Build the knowledge graph structure
def preprocess(fname):
    triples = {}

    ent_counter = 0
    rel_counter = 0
    train_counter = 0

    print (fname)
    #gzfile= gzip.open(fname, mode='rt')

    for line in csv.reader(open(fname), delimiter='\t', quotechar='"'):
        #print (line)
        h = line[0]
        r = line[1]
        t = line[2]
        
        train_counter +=1

        addTriple(triples, h, t, r)
        train_counter+=1
    print ('Triple:',train_counter)
    return triples

In [13]:
file = 'film.yago.graph'
triples = preprocess(file)

film.yago.graph
Triple: 593906


In [14]:
entities = list(triples.keys())
vocabulary = entities
print (len(vocabulary))

18423


In [15]:
vocabulary[:10]

['<The_Invisible_Army>',
 '<James_Pandu>',
 '<Stavisky>',
 '<Liberty_(1929_film)>',
 '<Macbeth_(2006_film)>',
 '<Tunisian_Victory>',
 "<Billy's_Love_Making>",
 '<Dog_Days_of_Summer_(film)>',
 '<A_Perfect_Place>',
 '<Psychomania>']

# Do random walks on the knowledge graph

In [16]:
def randomNWalkUniform(triples, n, walks, path_depth):
    path=[]
    for k in range(walks):
        walk = randomWalkUniform(triples, n, path_depth)
        path.append(walk)
    return path

In [17]:
walks = 20
path_depth = 5
paths = randomNWalkUniform(triples, entities[0], walks, path_depth)
print(paths)

[['<The_Invisible_Army>', 'rdf:type', '<wikicat_Black-and-white_films>', ''], ['<The_Invisible_Army>', '-<actedIn>', '<Bodil_Kjer>', ''], ['<The_Invisible_Army>', 'rdf:type', '<wikicat_Danish-language_films>', ''], ['<The_Invisible_Army>', 'rdf:type', '<wikicat_Danish_films>', ''], ['<The_Invisible_Army>', 'rdf:type', '<wikicat_Danish_films>', ''], ['<The_Invisible_Army>', '-<actedIn>', '<Bodil_Kjer>', ''], ['<The_Invisible_Army>', 'rdf:type', '<wikicat_Danish-language_films>', ''], ['<The_Invisible_Army>', 'rdf:type', '<wikicat_Black-and-white_films>', ''], ['<The_Invisible_Army>', '-<directed>', '<Johan_Jacobsen>', ''], ['<The_Invisible_Army>', '-<directed>', '<Johan_Jacobsen>', ''], ['<The_Invisible_Army>', '-<directed>', '<Johan_Jacobsen>', ''], ['<The_Invisible_Army>', 'rdf:type', '<wikicat_Danish_films>', ''], ['<The_Invisible_Army>', 'rdf:type', '<wikicat_Films_directed_by_Johan_Jacobsen>', ''], ['<The_Invisible_Army>', '-<directed>', '<Johan_Jacobsen>', ''], ['<The_Invisible_Ar

In [18]:

walks = 20
path_depth = 5

start_time =time.time()
sentences =[]
for word in vocabulary:
    sentences.extend( randomNWalkUniform(triples, word, walks, path_depth) )
elapsed_time = time.time() - start_time
print ('Time elapsed to generate features:',time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

Time elapsed to generate features: 00:00:03


In [19]:
model1 = gensim.models.Word2Vec(size=100, workers=8, window=5, sg=1, negative=15, iter=5)
model1.build_vocab(sentences)

In [20]:
corpus_count = model1.corpus_count
model1.train(sentences, total_examples = corpus_count, epochs = 5)

(3822630, 7369200)

In [21]:
model1.wv.most_similar('<The_Invisible_Army>')


/opt/conda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('<Cut_(2000_film)>', 0.9977931976318359),
 ('<Rossana_(film)>', 0.9973748326301575),
 ('<Bride_of_Frankenstein>', 0.9970694780349731),
 ('<Fata_Morgana_(1965_film)>', 0.9969465732574463),
 ('<D_(film)>', 0.996890127658844),
 ('<Carandiru_(film)>', 0.9967362880706787),
 ('<The_Loners>', 0.9967209100723267),
 ('<Plague_(1978_film)>', 0.996701717376709),
 ('<De_blaa_drenge>', 0.9966665506362915),
 ('<The_Terrible_People>', 0.9966117739677429)]